In [1]:
# SET UP
import pandas as pd
import altair as alt
import numpy as np
import folium, os
pd.set_option('display.max_columns', None)

## what years do you want to look back over (what is the 'lookback period')?
years = ['2016', '2017', '2018', '2019']
min_year = min(years) # <- find the earliest year (this is only used for chart titles later on)
years = pd.DataFrame(years)
years.columns = ['YEAR']

In [2]:
# ASSESSMENT DATA (using this as master list of all properties in Buffalo)
## read in raw csv
asmt = pd.read_csv('/home/dan/Python/QueenCityCounts/llrd_code/data/2019-2020_Assessment_Roll.csv', dtype=object)
## drop unusable columns and incomplete rows
asmt = asmt[['PRINT KEY','PROPERTY CLASS','NEIGHBORHOOD', 'HOUSE NUMBER', 'STREET']]
asmt.dropna(axis='index', how="any", inplace=True)
asmt = asmt[(asmt['NEIGHBORHOOD']!='UNKNOWN') | (asmt['NEIGHBORHOOD']!='')]
## format house number column (dropping all '0' and addresses higher than '9000', since those appear erroneous)
## NOTE: Assessment_Roll.csv (original OpenDataBuffalo data file) has no mention of '1/2' addresses
asmt['HOUSE NUMBER'] = pd.to_numeric(asmt['HOUSE NUMBER'], errors='coerce')
asmt = asmt[(asmt['HOUSE NUMBER']!=0) & (asmt['HOUSE NUMBER']<=9000)]
asmt['HOUSE NUMBER'] = asmt['HOUSE NUMBER'].astype(str)
## drop any duplicate rows
asmt.drop_duplicates(inplace=True)
## rename column headers
asmt.rename(columns={'PRINT KEY':'SBL','PROPERTY CLASS':'PROP_TYPE',\
                     'NEIGHBORHOOD':'NBHD', 'HOUSE NUMBER':'NUMBER'},inplace=True)
## concatenate 'NUMBER' and 'STREET' columns into one column
asmt['ADDRESS'] = asmt[['NUMBER','STREET']].apply(lambda x: ' '.join(x.values.astype(str)),axis=1)


In [3]:
asmt.head()
## NOTE: if the property is an apt building with multiple units, it will show up in the below
##       dataframe as multiple SBLs with the same ADDRESS (for example, '1088 DELAWARE')

,SBL,PROP_TYPE,NBHD,NUMBER,STREET,ADDRESS
1,123.77-7-28,210,Hopkins-Tifft,258,MYSTIC,258 MYSTIC
2,112.21-4-10.11,220,Broadway Fillmore,78,SOBIESKI,78 SOBIESKI
3,77.60-4-1,210,West Hertel,124,NORRIS,124 NORRIS
5,110.60-2-4.3,590,Lower West Side,10,SEVENTH NEW,10 SEVENTH NEW
6,78.56-5-9,220,North Park,218,COMMONWEALTH,218 COMMONWEALTH


In [4]:
# CODE VIOLATIONS DATA
## read in raw csv
vios = pd.read_csv('/home/dan/Python/QueenCityCounts/llrd_code/data/Code_Violations.csv', dtype=object)
## drop unnecessary columns, remove incomplete and/or duplicate rows
vios = vios[['DATE', 'UNIQUEKEY', 'ADDRESS']].drop_duplicates()
vios.dropna(axis='index', how="any", inplace=True)
## format the 'DATE' column, extract the 'YEAR' to a separate column
vios['DATE'] = vios['DATE'].apply(lambda x: str(x).split(' ')[0])
vios['DATE'] = pd.to_datetime(vios['DATE'])
vios['YEAR'] = vios['DATE'].apply(lambda x: str(x.year))
## split the 'ADDRESS' column into a 'NUMBER' and a 'STREET' column
vios['NUMBER'] = vios['ADDRESS'].apply(lambda x: str(x).split(' ')[0])
vios['STREET'] = vios['ADDRESS'].apply(lambda x: ' '.join(str(x).split(' ')[1:]))
## get a count of total violations, per address, per year 
## NOTE: defining "total violations" as count of unique UNIQUEKEYs at the address throughout the year
vios.rename(columns={'UNIQUEKEY':'VIOLATIONS'},inplace=True)
vios = pd.pivot_table(vios, index=['NUMBER', 'STREET', 'YEAR'], values=['VIOLATIONS'],
                      aggfunc=lambda x: len(x.unique()))
vios.reset_index(inplace=True)

In [5]:
vios.head()

,NUMBER,STREET,YEAR,VIOLATIONS
0,1,ALICE,2019,1
1,1,ALMA,2019,1
2,1,ARGYLE,2017,1
3,1,BUFFALO RIVER PLACE,2017,1
4,1,BUFFALO RIVER PLACE,2019,1


In [6]:
## RENTAL PROPERTY DATA
## read in raw csv
rent = pd.read_csv('/home/dan/Python/QueenCityCounts/llrd_code/data/Rental_Registry.csv', dtype=object)
## drop unnecessary columns, remove incomplete and/or duplicate rows
rent = rent[['Print Key', 'Address', 'License Status', 'Issued Datetime', 'Expiration Datetime']].drop_duplicates()
rent.dropna(axis='index', how="any", inplace=True)
## only procede with 'Active' records
rent = rent[rent['License Status']=='Active']
## concatenate 'NUMBER' and 'STREET' columns into one column
rent['NUMBER'] = rent['Address'].apply(lambda x: str(x).split(' ')[0])
rent['STREET'] = rent['Address'].apply(lambda x: ' '.join(str(x).split(' ')[1:]))
## rename column headers
rent.rename(columns={'License Status':'STATUS','Issued Datetime':'ISSUED',\
                     'Expiration Datetime':'EXPIRES','Print Key':'SBL','Address':'ADDRESS'}, inplace=True)
## change date columns from string to datetime datatypes
rent['ISSUED'] = pd.to_datetime(rent['ISSUED'])
rent['EXPIRES'] = pd.to_datetime(rent['EXPIRES'])
## create flag column
rent['IS_RENTAL'] = int(1)

In [7]:
rent.head()

,SBL,ADDRESS,STATUS,ISSUED,EXPIRES,NUMBER,STREET,IS_RENTAL
1,79.79-7-11,45 DUNLOP,Active,2019-01-18,2020-03-01,45,DUNLOP,1
2,99.67-7-10,188 NORMAL,Active,2019-02-15,2020-03-01,188,NORMAL,1
3,79.80-7-15,151 DARTMOUTH,Active,2019-02-20,2020-03-01,151,DARTMOUTH,1
4,79.72-3-12,371 MINNESOTA,Active,2019-02-20,2020-03-01,371,MINNESOTA,1
5,100.75-5-38.1,399 GUILFORD,Active,2018-12-26,2019-03-01,399,GUILFORD,1


In [21]:
# JOIN IT ALL TOGETHER (asmt + rent + vios)
## NOTE: OpenDataBuffalo reports Rental Status is by SBL, but Code Violations are reported by street address
##       repeat asmt dataframe for each year
years = years.assign(key=1)
asmt = asmt.assign(key=1)
df = asmt.merge(years, on='key',how='inner').drop(columns=['key','ADDRESS'])
## asmt <- rental (flag any sbl that is also a rental property)
## NOTE: the rental registry only has serious, reliable data back to 2018, so for the sake of this analysis,
##       let's pretend if its ever been a rental, it's been one for the entire 'lookback period'
df = df.merge(rent[['SBL', 'IS_RENTAL']].drop_duplicates(), on='SBL', how='left')
df['IS_RENTAL'].fillna(0, inplace=True)
## asmt+rental <- vios (how many violations were logged at that address, in that year)
df = df.merge(vios, on=['NUMBER','STREET','YEAR'], how='left')
df['VIOLATIONS'].fillna(0, inplace=True)
## organize dataframe (which now is asmt*year+rental+vios) and reset the index
df = df.sort_values(['YEAR','NUMBER','STREET'], ascending=True).reset_index(drop=True)

In [22]:
df.head()

,SBL,PROP_TYPE,NBHD,NUMBER,STREET,YEAR,IS_RENTAL,VIOLATIONS
0,666.00-60-1,831,Masten Park,1,AA,2016,0.0,0.0
1,89.57-4-3,210,Parkside,1,AGASSIZ,2016,0.0,0.0
2,90.33-4-4,210,Kensington-Bailey,1,ALICE,2016,0.0,0.0
3,90.64-4-1,482,Kenfield,1,ALMA,2016,0.0,0.0
4,122.35-2-5,281,Seneca Babcock,1,ANDREW,2016,1.0,0.0


In [23]:
# AGGREGATE BY NEIGHBORHOOD
## filter by 'PROP_TYPE' in the 200s
pt = df[(df['PROP_TYPE'].apply(lambda x: x[0])=='2')]
## drop unnecessary columns, remove duplicate rows
pt = pt[['SBL','NBHD','IS_RENTAL','VIOLATIONS']].drop_duplicates()
## for each sbl, are they on the rental registry, and have they had any citations during the lookback period 
pt = pd.pivot_table(pt, index=['NBHD', 'SBL'], values=['VIOLATIONS','IS_RENTAL'],
                    aggfunc={'IS_RENTAL':sum,'VIOLATIONS':sum}).reset_index()
pt['IS_RENTAL'] = pt['IS_RENTAL'].apply(lambda x: 'NO' if x==0.0 else 'YES')
pt['VIOLATIONS'] = pt['VIOLATIONS'].apply(lambda x: 'NO' if x==0.0 else 'YES') # <- at least one violation counts
## for each neighborhood, count sbls by rental/nonrental, and vios/no-vios 
pt = pd.pivot_table(pt, index=['NBHD', 'IS_RENTAL'], columns=['VIOLATIONS'], values=['SBL'], 
                    aggfunc={'SBL':len}).reset_index()
## as percents
pt['NO_VIOS'] = pt['SBL']['NO']/(pt['SBL']['NO'] + pt['SBL']['YES'])
pt['VIOS'] = pt['SBL']['YES']/(pt['SBL']['NO'] + pt['SBL']['YES'])
## reset column indexs headers
pt = pt.T.reset_index(drop=True).T
## rename column headers
pt.rename(columns={0:'NBHD',1:'IS_RENTAL',2:'NO_VIOS_RAW',3:'VIOS_RAW',4:'NO_VIOS_PCT',5:'VIOS_PCT'},inplace=True)
## calculate totals
pt.fillna(0, inplace=True)
pt['TOTAL'] = pt['NO_VIOS_RAW'] + pt['VIOS_RAW']
pt['TOTAL'] = pt['TOTAL'].apply(int)

In [24]:
pt.head()

,NBHD,IS_RENTAL,NO_VIOS_RAW,VIOS_RAW,NO_VIOS_PCT,VIOS_PCT,TOTAL
0,Allentown,NO,486.0,22.0,0.956693,0.043307,508
1,Allentown,YES,99.0,6.0,0.942857,0.057143,105
2,Black Rock,NO,457.0,98.0,0.823423,0.176577,555
3,Black Rock,YES,210.0,156.0,0.573770,0.426230,366
4,Broadway Fillmore,NO,1680.0,362.0,0.822723,0.177277,2042


In [25]:
# CHARTS
## date prep
## drop unnecessary columns, and wide to long existing pt
alt_pt = pt.drop(columns=['VIOS_RAW','NO_VIOS_RAW']).melt(id_vars=['NBHD','IS_RENTAL','TOTAL'])
## recast rental flag to 'RENT' for rental property and 'OO' for owner-occupied
alt_pt['TYPE'] = alt_pt['IS_RENTAL'].apply(lambda x: 'RENT' if x=='YES' else 'OO')
alt_pt.drop(columns=['IS_RENTAL'],inplace=True)
## rename column headers
alt_pt.rename(columns={'value':'PERCENT','variable':'STATUS'},inplace=True)
## nans to zeros
alt_pt.fillna(0, inplace=True)

## iterate through prepared data, and make charts
charts = {}
for nbhd in alt_pt['NBHD'].unique():
    tmp = alt_pt[(alt_pt['NBHD']==nbhd) & (alt_pt['STATUS']=='VIOS_PCT')]
    title = [nbhd, ('OO: ' + str(format(int(tmp[(tmp['NBHD']==nbhd) & (tmp['TYPE']=='OO')]['TOTAL']),',d'))\
                + ' | RENT: ' + str(format(int(tmp[(tmp['NBHD']==nbhd) & (tmp['TYPE']=='RENT')]['TOTAL']),',d')))]
    chart = alt.Chart(tmp[['NBHD','STATUS','TYPE','PERCENT']], title = title).mark_bar().encode(
        x=alt.X('TYPE', sort=['OO','RENT'], title = 'type of residence'),
        y=alt.Y('PERCENT:Q', axis=alt.Axis(format='%', title = '% that received at least one citation since ' + min_year), scale=alt.Scale(domain=(0, 1))),
        color=alt.Color('STATUS:N', legend = None,
                        scale=alt.Scale(
                            domain=['NO_VIOS_PCT','VIOS_PCT'],
                            range=['lightblue','red'])),
        order=alt.Order('TYPE')
    )
    charts.update({nbhd:chart})

## read out charts dictionary for html outfile
((charts['Allentown'] | charts['Black Rock'] | charts['Broadway Fillmore'] | charts['Central'] | charts['Central Park']) &
(charts['Delavan Grider'] | charts['Ellicott'] | charts['Elmwood Bidwell'] | charts['Elmwood Bryant'] | charts['Fillmore-Leroy']) &
(charts['First Ward'] | charts['Fruit Belt'] | charts['Genesee-Moselle'] | charts['Grant-Amherst'] | charts['Hamlin Park']) &
(charts['Hopkins-Tifft'] | charts['Kaisertown'] | charts['Kenfield'] | charts['Kensington-Bailey'] | charts['Lovejoy']) &
(charts['Lower West Side'] | charts['Masten Park'] | charts['MLK Park'] | charts['North Park'] | charts['Parkside']) &
(charts['Pratt-Willert'] | charts['Riverside'] | charts['Schiller Park'] | charts['Seneca Babcock'] | charts['Seneca-Cazenovia']) &
(charts['South Park'] | charts['University Heights'] | charts['Upper West Side'] | charts['West Hertel'] | charts['West Side'])).save('/home/dan/Python/QueenCityCounts/llrd_code/charts.html')

In [13]:
# MAP
import folium, os
## find geojson file
geojson = os.path.join('data','Neighborhoods.geojson')
## initialize map
m = folium.Map([42.900155, -78.8485], zoom_start=12)
## overlay geojson to map object
folium.GeoJson(geojson, \
               tooltip = folium.GeoJsonTooltip(fields=['nbhdname'],labels = False)\
              ).add_to(m)
## outfile to html
m.save('/home/dan/Python/QueenCityCounts/llrd_code/map.html')